In [1]:
import os
from pathlib import Path

cwd = Path(os.getcwd())

In [2]:
action_plays = cwd / "clips" / "action_plays"
off_plays = cwd / "clips" / "off_plays"
whistles = cwd / "clips" / "whistles"

data_dirs = [action_plays, off_plays, whistles]
data_files = []

for data_dir in data_dirs:    
    filenames = [str(data_dir) + "/" + f for f in os.listdir(data_dir)]
    data_files.extend(filenames)

data_files = [item for item in data_files if ".DS_Store" not in item] # remove .DS_Store file
print(data_files)

['/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip2b.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip2a.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip9a.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip10a.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip8d.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip5.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip9b.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip10b.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip10c.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip4.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/clip8c.wav', '/Users/prathikr/Desktop/most-valuable-video-player/clips/action_plays/cli

In [3]:
import torchaudio

# check all files have 2 channels and 44.1k sampling rate

for file in data_files:
    sig, sr = torchaudio.load(file)
    print(sig.shape[0], sr, end=" ")

2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 1 44100 1 44100 1 44100 1 44100 1 44100 1 44100 1 44100 1 44100 2 44100 1 44100 2 44100 2 44100 1 44100 1 44100 1 44100 1 44100 2 44100 2 44100 2 44100 2 44100 2 44100 1 44100 1 44100 1 44100 1 44100 2 44100 2 44100 2 44100 2 44100 2 44100 1 44100 2 44100 1 44100 2 44100 1 44100 1 44100 2 44100 1 44100 1 44100 1 44100 1 44100 1 44100 2 44100 2 44100 1 44100 1 44100 1 44100 2 44100 1 44100 1 44100 2 44100 2 44100 1 44100 1 44100 2 44100 1 44100 1 44100 2 44100 2 44100 2 44100 1 44100 2 44100 1 44100 2 44100 1 44100 1 44100 1 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 2 44100 

In [4]:
import math, random
import torch
from torchaudio import transforms
from IPython.display import Audio

class AudioUtil():
    # ----------------------------
    # Load an audio file. Return the signal as a tensor and the sample rate
    # ----------------------------
    @staticmethod
    def open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)
            
    # ----------------------------
    # Convert the given audio to the desired number of channels
    # ----------------------------
    @staticmethod
    def rechannel(aud, new_channel):
        sig, sr = aud

        if (sig.shape[0] == new_channel):
            # Nothing to do
            return aud

        if (new_channel == 1):
            # Convert from stereo to mono by selecting only the first channel
            resig = sig[:1, :]
        else:
            # Convert from mono to stereo by duplicating the first channel
            resig = torch.cat([sig, sig])

        return ((resig, sr))

    # ----------------------------
    # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
    # ----------------------------
    @staticmethod
    def pad_trunc(aud, max_ms):
        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr//1000 * max_ms

        if (sig_len > max_len):
            # Truncate the signal to the given length
            sig = sig[:,:max_len]

        elif (sig_len < max_len):
            # Length of padding to add at the beginning and end of the signal
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len

            # Pad with 0s
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))

            sig = torch.cat((pad_begin, sig, pad_end), 1)

        return (sig, sr)

    # ----------------------------
    # Shifts the signal to the left or right by some percent. Values at the end
    # are 'wrapped around' to the start of the transformed signal.
    # ----------------------------
    @staticmethod
    def time_shift(aud, shift_limit):
        sig,sr = aud
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return (sig.roll(shift_amt), sr)

    # ----------------------------
    # Generate a Spectrogram
    # ----------------------------
    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig,sr = aud
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return (spec)

    # ----------------------------
    # Augment the Spectrogram by masking out some sections of it in both the frequency
    # dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent
    # overfitting and to help the model generalise better. The masked sections are
    # replaced with the mean value.
    # ----------------------------
    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec

In [5]:
from torch.utils.data import DataLoader, Dataset, random_split

# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
    def __init__(self, data_files):
        self.data_files = data_files
        self.duration = 5000
        self.sr = 44100
        self.channel = 2
        self.shift_pct = 0.4

    # ----------------------------
    # Number of items in dataset
    # ----------------------------
    def __len__(self):
        return len(self.data_files)    

    # ----------------------------
    # Get i'th item in dataset
    # ----------------------------
    def __getitem__(self, idx):
        # Absolute file path of the audio file - concatenate the audio directory with
        # the relative path
        audio_file = self.data_files[idx]
        # Get the Class ID
        class_id = -1
        if "whistle" in audio_file:
            class_id = 0
        elif "action_play" in audio_file:
            class_id = 1
        elif "off_play" in audio_file:
            class_id = 2

        aud = AudioUtil.open(audio_file)

        rechan = AudioUtil.rechannel(aud, self.channel)
        dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
        shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
        sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
        aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

        return aug_sgram, class_id

In [6]:
from torch.utils.data import random_split

myds = SoundDS(data_files)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

In [7]:
print("setting hardware backend...")
if torch.cuda.is_available():
    device=torch.device('cuda')
elif torch.backends.mps.is_available():
    device=torch.device('mps')
else:
    device=torch.device('cpu')
print(f"{device} backend set")

setting hardware backend...
mps backend set


In [8]:
import torch.nn.functional as F
from torch.nn import init
import torch.nn as nn

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=3)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        print(x.shape)
        # Linear layer
        x = self.lin(x)
        x = nn.functional.softmax(x, dim=1)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel = AudioClassifier()
myModel = myModel.to(device)

In [9]:
# ----------------------------
# Training Loop
# ----------------------------
def training(model, train_dl, num_epochs):
    # Loss Function, Optimizer and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                            steps_per_epoch=int(len(train_dl)),
                            epochs=num_epochs,
                            anneal_strategy='linear')

    # Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

        # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Keep stats for Loss and Accuracy
            running_loss += loss.item()

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction/total_prediction
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

    print('Finished Training')

num_epochs=25   # Just for demo, adjust this higher.
training(myModel, train_dl, num_epochs)

torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([14, 64])
Epoch: 0, Loss: 1.10, Accuracy: 0.36
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([14, 64])
Epoch: 1, Loss: 1.07, Accuracy: 0.53
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([14, 64])
Epoch: 2, Loss: 1.03, Accuracy: 0.65
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
torch.S

KeyboardInterrupt: 

In [ ]:
aud0 = AudioUtil.open("/Users/prathikr/Desktop/volleyball-audio-data/clips/whistles/clip12.wav") # class 0
aud1 = AudioUtil.open("/Users/prathikr/Desktop/volleyball-audio-data/clips/action_plays/clip2b.wav") # class 1
aud2 = AudioUtil.open("/Users/prathikr/Desktop/volleyball-audio-data/clips/off_plays/clip9.wav") # class 2

dur_aud0 = AudioUtil.pad_trunc(aud0, 5000)
shift_aud0 = AudioUtil.time_shift(dur_aud0, 0.4)
sgram0 = AudioUtil.spectro_gram(shift_aud0, n_mels=64, n_fft=1024, hop_len=None)
aug_sgram0 = AudioUtil.spectro_augment(sgram0, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

dur_aud1 = AudioUtil.pad_trunc(aud1, 5000)
shift_aud1 = AudioUtil.time_shift(dur_aud1, 0.4)
sgram1 = AudioUtil.spectro_gram(shift_aud1, n_mels=64, n_fft=1024, hop_len=None)
aug_sgram1 = AudioUtil.spectro_augment(sgram1, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

dur_aud2 = AudioUtil.pad_trunc(aud2, 5000)
shift_aud2 = AudioUtil.time_shift(dur_aud2, 0.4)
sgram2 = AudioUtil.spectro_gram(shift_aud2, n_mels=64, n_fft=1024, hop_len=None)
aug_sgram2 = AudioUtil.spectro_augment(sgram2, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

test_dl = torch.utils.data.DataLoader([aug_sgram0, aug_sgram1, aug_sgram2], batch_size=3, shuffle=False)

# Get predictions
for test in test_dl:
    test = test.to(device)
    outputs = myModel(test)
    # Get the predicted class with the highest score
    _, prediction = torch.max(outputs,1)
    print(prediction)

In [ ]:
# ----------------------------
# Inference
# ----------------------------
def inference (model, val_dl):
    correct_prediction = 0
    total_prediction = 0

    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            print("prediction:", prediction)
            print("labels:    ", labels)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction/total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

# Run inference on trained model with the validation set
inference(myModel, val_dl)

In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks
import re
import shutil

output_dir = cwd / "tmp"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

myaudio = AudioSegment.from_file("los-altos-comeback.wav" , "wav") 
chunk_length_ms = 2500 # pydub calculates in millisec
chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

for i, chunk in enumerate(chunks):
    chunk_name = "chunk{0}.wav".format(i)
    chunk.export(output_dir / chunk_name, format="wav")

mypath = "tmp/"
chunk_files = [os.path.join(dirpath,f) for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames]
chunk_files.sort(key=lambda test_string : list(map(int, re.findall(r'\d+', test_string)))[0]) # sort by chunk number
print(chunk_files)

In [ ]:
test_data = []

for data_file in chunk_files:
    full_path = cwd / data_file
    aud = AudioUtil.open(full_path)

    dur_aud = AudioUtil.pad_trunc(aud, 5000)
    shift_aud = AudioUtil.time_shift(dur_aud, 0.4)
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    test_data.append(aug_sgram)
    
test_dl = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False)
# Get predictions
for i, data in enumerate(test_dl):
    inputs = data.to(device)
    
    # Normalize the inputs
    inputs_m, inputs_s = inputs.mean(), inputs.std()
    inputs = (inputs - inputs_m) / inputs_s
        
    outputs = myModel(inputs)
    # Get the predicted class with the highest score
    _, prediction = torch.max(outputs,1)
    print(outputs)